In [ ]:
import telebot
import json
import feedparser
import re
import random
from bs4 import BeautifulSoup


token = '980878523:AAEUYxMijMvEJTuS55crcbQc10JLbF_ApM4'
bot = telebot.TeleBot(token)
galochka = '✔'
last_click = None

# стартовая клавиатура
start_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
start_kbrd.row('Новости', 'Анекдот')
start_kbrd.row('Новый источник')
start_kbrd.row('Категория', 'Ключевые слова')
start_kbrd.row('Выход')

# клавиатура с выбором категорий
categories_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
categories_kbrd.row('Политика', 'Спорт')
categories_kbrd.row('Наука', 'Искусство')
categories_kbrd.row('Отмена')

# клавиатура общего назначения (отмены)
cancel_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
cancel_kbrd.row('Отмена')


@bot.message_handler(commands=['start', 'help'])
def start_message(message):
    global start_kbrd

    bot.send_message(message.chat.id,
                     'привет!  :)\n\n' +
                     'я тебе каких-нибудь жутко нерелевантных новостей покидаю, ок?', 
                     reply_markup=start_kbrd)
    
    # добавление нового пользователя в json-файл
    add_user(message.from_user.id)
    
    
@bot.message_handler(content_types=['text'])
def send_message(message):
    global start_kbrd, categories_kbrd, cancel_kbrd, last_click
    
    if message.text == 'Новости':
        last_click = None
        bot.send_message(message.chat.id, 'типа высылаю тебе новости, окда')
    
    elif message.text == 'Анекдот':
        last_click = None
        bot.send_message(message.chat.id, random_joke())
    
    elif message.text == 'Новый источник':
        last_click = 'add source'
        bot.send_message(message.chat.id, 'введи ссылку на rss/atom ленту и ее категорию в формате:' +
                        '\n[ссылка] [категория]', reply_markup=cancel_kbrd)
    
    elif message.text == 'Категория':
        last_click = 'choose category'
        bot.send_message(message.chat.id, 'выбери что-нибудь из этого: ', reply_markup=categories_kbrd)
    
    elif message.text == 'Ключевые слова':
        last_click = 'choose keywords'
        bot.send_message(message.chat.id, 'введи ключевые слова через запятую.' + '\nнапример: Чемпионат конфликт беспорядки',
                        reply_markup=cancel_kbrd)
        
    elif message.text == 'Выход':
        last_click = None
        bot.send_message(message.chat.id, 'много будешь знать - скоро состаришься')
    
    # обработка перехода к предыдущему шагу
    elif message.text == 'Отмена':
        if last_click == 'choose category':
            bot.send_message(message.chat.id, 'ты отменил выбор категории. \nвыбери дальнейшее действие.',
                             reply_markup=start_kbrd)
        elif last_click == 'choose keywords':
            bot.send_message(message.chat.id, 'ты отменил выбор ключевых слов. \nвыбери дальнейшее действие.',
                             reply_markup=start_kbrd)
        elif last_click == 'add source':
            bot.send_message(message.chat.id, 'ты отменил добавление нового источника. \nвыбери дальнейшее действие.',
                             reply_markup=start_kbrd)
    
    else:
        if last_click == 'add source':
            add_source(message.text)
        elif last_click == 'choose category':
            add_preferences(message.from_user.id, 'categories', message.text)
        elif last_click == 'choose keywords':
            add_preferences(message.from_user.id, 'keywords', message.text)
        
        bot.send_message(message.chat.id, 'что будем делать дальше?', reply_markup=start_kbrd)
        last_click = None
    
# новый пользователь    
def add_user(user_id):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
        
    if str(user_id) not in list(current['users'].keys()):
        current['users'][str(user_id)] = {"categories": [], "keywords": []}
    
    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)
    
# новые предпочтения пользователя (ключевые слова/категория)
def add_preferences(user_id, data_type, data):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    user_id = str(user_id)
    if data_type == 'categories':
        for word in data.lower().split():
            if word in current['sources'].keys():
                if word not in current['users'][user_id][data_type]:
                    current['users'][user_id][data_type].append(word)
        
    elif data_type == 'keywords':
        for word in data.lower().split():
            if word not in current['users'][user_id][data_type]:
                current['users'][user_id][data_type].append(word)
      
    
    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)
    
# новый источник
def add_source(data):
    global bot
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    link, category = data.lower().split()
    
    if link[-3::] == 'xml':
        
        if category in current['sources'].keys():
            if link not in current['sources'][category]:
                current['sources'][category].append(link)
        else:
            current['sources'][category] = [link]

    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)

# очистка самой новости от тэгов и прочего мусора
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return ' '.join(cleantext.split())
        
    
def random_joke():
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    link = random.choice(current['sources']['joke'])
    jokes = feedparser.parse(link).entries
    
    return cleanhtml(jokes[random.randint(0, len(jokes) - 1)].summary)


bot.polling()